Script de medición y borrado de las colecciones en cluster MongoDB
TFM Daniel Herranz Segundo

In [2]:
import pandas as pd
import numpy as np
import os
import json
import random
import pymongo
from pymongo import MongoClient
import time
from pprint import pprint
from pymongo import InsertOne, DeleteMany, ReplaceOne, UpdateOne
import matplotlib.pyplot as plt
import psutil

In [3]:
def save_results_to_csv(results,file):
    #Guardamos los resultados en csv
    from datetime import datetime
    dia = datetime.now().strftime("%d%m%Y_%H_%M_%S") 
    data = results
    results_df = pd.DataFrame(data, columns =['Registros', 'Tiempo', 'CPU','Memoria'])
    results.to_csv(file.format(str(dia), write_concern))

In [4]:
resultados_etl = '../Results/MongoDB/MongoDB_Delete_MultipleCollection_{}_wc_{}.csv'

In [5]:
#repeticiones
repeats = 1000

In [6]:
write_concern = 3

#Conexion unica a colecciones
connection = MongoClient('localhost', 27017, w=3) #Conexion con WriteConcern a 3 (primario y dos nodos consolidados)

#Creamos una base de datos para el trabajo llamada TFM
tfm_mongo_db = connection["tfm_mongo_database"]

## Creación de las colecciones por cada dominio

In [7]:
CustomerProfileCollection = tfm_mongo_db["CustomerProfileCollection"]
CurrentAccountCollection = tfm_mongo_db["CurrentAccountCollection"]
PositionKeepingCollection = tfm_mongo_db["PositionKeepingCollection"]
print(connection.list_database_names())

['admin', 'config', 'local', 'tfm_mongo_database']


## Borrado de los documentos y simulación en cascada

In [8]:
def removePartyIdInDifferentsDomains(partyId):
    #Baja de un cliente -Borrado de todos los documentos relacionados con PartyId (CurrentAccount + CustomerProfile + PositionKeeping (AccountId))
    currents_accounts_to_delete = []
    resultado = CurrentAccountCollection.find({"PartyId": partyId})
    for doc in resultado:
        # Borramos todos los registros de Balance asociados a cuentas de ese PartyId
        currents_accounts_to_delete.append(doc.get('AccountId'))
    currents_accounts_to_delete = list(dict.fromkeys(currents_accounts_to_delete))
    for accountId_to_remove in currents_accounts_to_delete:
        PositionKeepingCollection.delete_many({"AccountId": accountId_to_remove})
        CustomerProfileCollection.delete_many({"PartyId" : partyId})

In [9]:
#Cargas Masiva con Many
def deletePartyIdInDatabase(partyIdList):
    registers = [] 
    for partyId in range(0,len(partyIdList)):      
        time_inicial = time.time()
        removePartyIdInDifferentsDomains(partyId)
        time_final = time.time()
        data_time_collection = round(time_final - time_inicial,3)
        used_cpu = psutil.cpu_percent()
        mem_used = psutil.virtual_memory().percent
        registers.append((CustomerProfileCollection.count_documents({}),data_time_collection,used_cpu,mem_used))
    return registers

In [14]:
#Seleccionamos todos los partyId existentes en la coleccion CustomerProfile para borrarlos iterativamente
partyIdList = []
resultado_query = CustomerProfileCollection.find({},{'PartyId': 1})
for customer in resultado_query:
    partyIdList.append(customer.get('PartyId'))
partyIdList = list(dict.fromkeys(partyIdList))
partyIdList = partyIdList[:100]
len(partyIdList)

100

In [15]:
#Estado actual de las diferentes colecciones
print("CustomerProfileCollection",CustomerProfileCollection.count_documents({}),"documentos antes de borrado")
print("CurrentAccountCollection",CurrentAccountCollection.count_documents({}),"documentos antes de borrado")
print("PositionKeepingCollection",PositionKeepingCollection.count_documents({}),"documentos antes de borrado")

CustomerProfileCollection 967347 documentos antes de borrado
CurrentAccountCollection 1000000 documentos antes de borrado
PositionKeepingCollection 967347 documentos antes de borrado


In [16]:
#Iteramos sobre partyId al azar (para evitar sesgo de indices) y borramos todo lo relacionado en otras colecciones
registers = deletePartyIdInDatabase(partyIdList)

KeyboardInterrupt: 

In [ ]:
data = registers
results_df = pd.DataFrame(data, columns =['Registros', 'Tiempo', 'CPU','Memoria'])
save_results_to_csv(results_df,resultados_etl)
results_df.sample(5)

In [ ]:
#Estado actual de las diferentes colecciones
print("CustomerProfileCollection",CurrentAccountCollection.count_documents({}),"documentos post borrado")
print("CurrentAccountCollection",CurrentAccountCollection.count_documents({}),"documentos post borrado")
print("PositionKeepingCollection",PositionKeepingCollection.count_documents({}),"documentos post borrado")

In [ ]:
#Cierre de la conexion
connection.close()